# Рекомендация тарифов

В нашем распоряжении данные о пользователях оператора мобильной связи Мегалайн, перешедшие на тарифы Смарт или Ультра. Необходимо построить модель для задачи классификации, которая выберет подходящий тариф.

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Исследование-модели-DecisionTreeClassifier" data-toc-modified-id="Исследование-модели-DecisionTreeClassifier-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Исследование модели <em>DecisionTreeClassifier</em></a></span></li><li><span><a href="#Исследование-модели-RandomForestClassifier" data-toc-modified-id="Исследование-модели-RandomForestClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Исследование модели <em>RandomForestClassifier</em></a></span></li><li><span><a href="#Исследование-модели-LogisticRegression" data-toc-modified-id="Исследование-модели-LogisticRegression-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Исследование модели <em>LogisticRegression</em></a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование-моделей" data-toc-modified-id="Тестирование-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование моделей</a></span><ul class="toc-item"><li><span><a href="#Проверка-модели-DecisionTreeClassifier" data-toc-modified-id="Проверка-модели-DecisionTreeClassifier-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Проверка модели <em>DecisionTreeClassifier</em></a></span></li><li><span><a href="#Проверка-модели-RandomForestClassifier" data-toc-modified-id="Проверка-модели-RandomForestClassifier-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Проверка модели <em>RandomForestClassifier</em></a></span></li><li><span><a href="#Проверка-модели-LogisticRegression" data-toc-modified-id="Проверка-модели-LogisticRegression-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Проверка модели <em>LogisticRegression</em></a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Обзор данных

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
warnings.filterwarnings('ignore')

RS=111

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Рассмотрим первые 5 строк исходных данных:

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Из документации к данным:
* **сalls** — количество звонков,
* **minutes** — суммарная длительность звонков в минутах,
* **messages** — количество sms-сообщений,
* **mb_used** — израсходованный интернет-трафик в Мб,
* **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Рассмотрим последние 5 строк исходных данных:

In [4]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


Рассмотрим случайные 5 строк исходных данных:

In [5]:
df.sample(5, random_state=RS)

,calls,minutes,messages,mb_used,is_ultra
1145,43.0,223.90,12.0,23090.88,0
1329,111.0,818.91,3.0,15530.91,1
1137,52.0,378.93,117.0,23449.04,1
1283,60.0,427.29,6.0,7023.77,0
1960,123.0,854.24,47.0,19055.86,1


Выведем основную информацию о таблице:

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Выведем описательные статистики:

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


### Вывод

Предварительно можно утверждать, что имеющихся данных достаточно для построение моделей.

## Разделение данных на выборки

In [8]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=RS)

In [9]:
df_test, df_valid = train_test_split(df_test, test_size = 0.5, random_state=RS)

In [10]:
df_train.shape

(1928, 5)

In [11]:
df_test.shape

(643, 5)

In [12]:
df_valid.shape

(643, 5)

In [13]:
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

### Вывод

Были разделены исходные данные на обучающую, тестовую и валидационную выборки в соотношении 3:1:1.

## Исследование моделей

Построим функции для каждой модели, возвращающую сообщение о значениях гиперпараметров, при которых достигается наиболее оптимальное значение **accuracy**:

In [14]:
def DTC_model_analysis():
    best_depth = 0
    best_acc = 0
    
    for depth in range(1, 25):
        model = DecisionTreeClassifier(max_depth=depth, random_state=RS)
        model.fit(train_features, train_target)
        predictions = model.predict(valid_features)
        accuracy = accuracy_score(valid_target, predictions)
            
        if accuracy > best_acc:
            best_acc = accuracy
            best_depth = depth
    
    return f'Для данного типа модели наилучшее значение accuracy равное {best_acc:.2f} достигается при глубине {best_depth}'

In [15]:
def RFC_model_analysis():
    best_depth = 0
    best_acc = 0
    best_est = 0
    
    for depth in range(1, 25):
        for est in range(1, 25):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=RS)
            model.fit(train_features, train_target)
            predictions = model.predict(valid_features)
            accuracy = accuracy_score(valid_target, predictions)
                
            if accuracy > best_acc:
                best_acc = accuracy
                best_depth = depth
                best_est = est
    
    return f'Для данного типа модели наилучшее значение accuracy равное {best_acc:.2f} достигается при глубине {best_depth} и количестве деревьев {best_est}'

In [16]:
def LR_model_analysis():
    model = LogisticRegression(random_state=RS)
    model.fit(train_features, train_target)
    predictions = model.predict(valid_features)
    accuracy = model.score(valid_features, valid_target)
        
    return f'Для данного типа модели значение accuracy равно {accuracy:.2f}' 

### Исследование модели *DecisionTreeClassifier*

In [17]:
DTC_model_analysis()

'Для данного типа модели наилучшее значение accuracy равное 0.78 достигается при глубине 10'

### Исследование модели *RandomForestClassifier*

In [18]:
RFC_model_analysis()

'Для данного типа модели наилучшее значение accuracy равное 0.80 достигается при глубине 11 и количестве деревьев 21'

### Исследование модели *LogisticRegression*

In [19]:
LR_model_analysis()

'Для данного типа модели значение accuracy равно 0.74'

### Вывод

Был проведен анализ следующих моделей:
* Decision Tree Classifier
* Random Forest Classifier
* Logistic Regression

Установлено, что наиболее оптимальное значение **accuracy = 0.8** достигается при использовании модели `Random Forest Classifier` при следующих гиперпараметрах: *max_depth = 11* и *n_estimators = 21*.

Для модели `Decision Tree Classifier` оптимальное значение **accuracy = 0.78** достигается при *max_depth = 10*.

Наименьшее значение **accuracy = 0.74** Было получено при использовании модели `Logistic Regression`.

## Тестирование моделей

### Проверка модели *DecisionTreeClassifier*

In [20]:
model = DecisionTreeClassifier(max_depth=10, random_state=RS)
model.fit(train_features, train_target)
predictions = model.predict(test_features)
accuracy = accuracy_score(test_target, predictions)

In [21]:
print(f'{accuracy:.2f}')

0.77


### Проверка модели *RandomForestClassifier*

In [22]:
model = RandomForestClassifier(max_depth=11, n_estimators=21, random_state=RS)
model.fit(train_features, train_target)
predictions = model.predict(test_features)
accuracy = accuracy_score(test_target, predictions)

In [23]:
print(f'{accuracy:.2f}')

0.80


### Проверка модели *LogisticRegression*

In [24]:
model = LogisticRegression(random_state=RS)
model.fit(train_features, train_target)
predictions = model.predict(test_features)
accuracy = model.score(test_features, test_target)

In [25]:
print(f'{accuracy:.2f}')

0.75


### Вывод

Проверка моделей на тестовой выборке показала, что значение **accuracy** совпадает со значениями для валидационной выборки.

## Проверка модели на адекватность

In [26]:
train_target.value_counts(normalize=True)

0    0.696577
1    0.303423
Name: is_ultra, dtype: float64

Для проверки модели на адекватность создадим константную модель:

In [27]:
dummy_model = DummyClassifier(strategy='constant', random_state=RS, constant=0)
dummy_model.fit(train_features, train_target)
predictions = dummy_model.predict(test_features)
accuracy = dummy_model.score(test_features, test_target)

In [28]:
accuracy

0.6951788491446346

Значение метрики выбранной модели больше чем у константной - следовательно, модель прошла проверку на адекватность.

## Общий вывод

Таким образом был проведен анализ моделей классификации. Наилучший результат показала модель `RandomForestClassifier` при гиперпараметрах *max_depth = 11* и *n_estimators = 21*. Ее **accuracy = 0.8**.